In [2]:
import numpy as np
from sklearn.neighbors import kneighbors_graph
from sklearn.datasets import make_moons

from scipy.sparse.csgraph import laplacian
from scipy.spatial.distance import pdist, squareform
from lund.lund import LearningbyUnsupervisedNonlinearDiffusion
from lund.utils import GraphExtractor,DensityEstimator
from model.utils import loadHSI,calculate_aligned_accuracy

# data_path, gt_path, data_name, gt_name = '/Users/seoli/Desktop/DIAMONDS/Tufts2024/data/SalinasA_corrected.mat', '/Users/seoli/Desktop/DIAMONDS/Tufts2024/data/SalinasA_gt.mat', 'salinasA_corrected', 'salinasA_gt'


# X, M, N, D, HSI, GT, Y, n, K = loadHSI(data_path, gt_path, data_name, gt_name)
# print(X.shape)
X, GT = make_moons(n_samples=1000, noise=0.1)

print("Data shape:", X.shape)

# GT = GT - 1
# HSI = X.reshape((M, N, D))

graph_extractor = GraphExtractor(sigma=1.0, DiffusionNN=10, NEigs=10) #should probably add like sucess print statements here bc idrk if this works just yet..
density_estimator = DensityEstimator(DensityNN=10, Sigma0=1.0)

p = density_estimator.kde(X)
G = graph_extractor.extract_graph(X)

t = 43

C, K, Dt = LearningbyUnsupervisedNonlinearDiffusion(X, t, G, p)



print("Cluster labels:", C)
print("Number of clusters:", K)
print("Diffusion distances:", Dt)

accuracy = calculate_aligned_accuracy(GT, C)
print("Aligned Accuracy:", accuracy)



Data shape: (1000, 2)
d is squareformed
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.05002042 0.03542763 0.00564436 ... 0.01513758 0.02412449 0.02531855]
 [0.05116843 0.04874425 0.02182114 ... 0.03364947 0.0576586  0.0327489 ]
 ...
 [2.26166459 1.79903096 1.88033143 ... 2.46803924 2.18613066 3.14015715]
 [2.26842281 1.80912744 1.902898   ... 2.4868095  2.19965193 3.14397907]
 [2.3304678  1.84941649 1.94894806 ... 2.53795002 2.22362193 3.23291146]]
pre sum)
Updated
Dist AFTER squareform: [[0.         1.41150352 0.61743948 ... 1.87643648 1.81906005 1.42310565]
 [1.41150352 0.         0.98162378 ... 0.73023402 0.43553007 1.59385387]
 [0.61743948 0.98162378 0.         ... 1.29451846 1.32477364 1.68657718]
 ...
 [1.87643648 0.73023402 1.29451846 ... 0.         0.4676459  2.32218543]
 [1.81906005 0.43553007 1.32477364 ... 0.4676459  0.         1.97487621]
 [1.42310565 1.59385387 1.68657718 ... 2.32218543 1.97487621 0.        ]]
distance proper init
entering tr

IndexError: boolean index did not match indexed array along dimension 0; dimension is 10 but corresponding boolean dimension is 1000